In [1]:
# import necessary torch libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random

# import MLP model
from CNN_architecture import VGG11

# seed values
seed = 42  # or any number you prefer
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [2]:
# transform the data to tensor and normalize it
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

num_workers = 4
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [3]:
# Model loading and other boilerplate code required to train the model
model = VGG11()

# Loss function and optimizer (PyTorch object initialization)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [4]:
# Helper functions to train model and visualise loss and accuracy
def reset_model(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

def plot_training_loss_and_accuracy(train_loss, train_acc, valid_loss, valid_acc):
    import matplotlib.pyplot as plt

    # Plotting the loss and accuracy curves during training
    fig, axes = plt.subplots(1, 2, figsize=(10,5))

    # Plot on the first subplot
    axes[0].grid(True, alpha=0.35, linestyle='-.', color='black', linewidth=0.75)
    axes[0].plot(range(1,len(train_loss)+1), train_loss, color='red', linestyle='-', linewidth=1, marker='.', label = 'Training loss')
    axes[0].plot(range(1, len(valid_loss)+1), valid_loss,  color='green', linestyle='-', linewidth=1, marker='.', label = 'Validation loss')
    axes[0].set_title('Loss Curves', fontsize=16)
    axes[0].set_xlabel('Epochs', fontsize=12)
    axes[0].legend()
    axes[0].set_ylabel('Average Loss over an epoch ', fontsize=11)

    # Plot on the second subplot
    axes[1].grid(True, alpha=0.35, linestyle='-.', color='black', linewidth=0.75)
    axes[1].plot(range(1, len(train_acc)+1), train_acc,  color='red', linestyle='-', linewidth=1, marker='.', label = 'Training Accuracy')
    axes[1].plot(range(1, len(valid_acc)+1), valid_acc,  color='green', linestyle='-', linewidth=1, marker='.', label = 'Validation Accuracy')
    axes[1].set_title('Accuracy Curves', fontsize=16)
    axes[1].set_xlabel('Epochs', fontsize=12)
    axes[1].legend()
    axes[1].set_ylabel('Average Accuracy over an epoch', fontsize=11)

    plt.tight_layout()  # Adjust spacing
    plt.show()

    return None

In [ ]:
import torch

def training_vgg11(model, loss_func, optimizer, trainloader, testloader, epochs=10, log_file="training_log.txt"):
    # Reset model weights
    # reset_model(model)

    # Initialize txt file with CSV-style headers
    with open(log_file, 'w') as file:
        file.write("Epoch,Train Loss,Train Accuracy,Validation Loss,Validation Accuracy\n")

    # Tracking lists
    train_losses = []
    validation_loss = []
    validation_accuracy = []
    train_accuracies = []

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        # Training phase
        for i, (inputs, labels) in enumerate(trainloader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        avg_loss = running_loss / total
        accuracy = 100 * correct / total
        train_losses.append(avg_loss)
        train_accuracies.append(accuracy)

        # Validation phase
        validation_error = 0.0
        total_test = 0
        correct_test = 0
        with torch.no_grad():
            for i, (inputs_test, labels_test) in enumerate(testloader):
                outputs_test = model(inputs_test)
                loss_test = loss_func(outputs_test, labels_test)
                validation_error += loss_test.item()
                total_test += labels_test.size(0)

                _, predicted_test = torch.max(outputs_test, dim=1)
                correct_test += (predicted_test == labels_test).sum().item()

        val_loss = validation_error / total_test
        val_accuracy = 100 * correct_test / total_test
        validation_loss.append(val_loss)
        validation_accuracy.append(val_accuracy)

        # Save metrics in CSV format to a .txt file
        with open(log_file, 'a') as file:
            file.write(f"{epoch + 1},{avg_loss:.4f},{accuracy:.2f}%,{val_loss:.4f},{val_accuracy:.2f}%\n")

        # Print progress
        print(f"Epoch [{epoch+1}/{epochs}], "
              f"Train Loss: {avg_loss:.4f}, Train Acc: {accuracy:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    print("Finished Training")

    return train_losses, train_accuracies, validation_loss, validation_accuracy

# train for 10 epochs
t_loss100, t_acc100, v_loss100, v_acc100 = training_vgg11(model, loss_func, optimizer, trainloader,testloader, epochs=100)
plot_training_loss_and_accuracy(t_loss100, t_acc100, v_loss100, v_acc100)

Epoch [1/100], Train Loss: 0.0360, Train Acc: 15.78%, Val Loss: 0.0360, Val Acc: 14.96%
Epoch [2/100], Train Loss: 0.0337, Train Acc: 23.27%, Val Loss: 0.0299, Val Acc: 30.95%
Epoch [3/100], Train Loss: 0.0277, Train Acc: 35.42%, Val Loss: 0.0258, Val Acc: 39.80%
Epoch [4/100], Train Loss: 0.0246, Train Acc: 42.49%, Val Loss: 0.0232, Val Acc: 45.20%
Epoch [5/100], Train Loss: 0.0229, Train Acc: 46.85%, Val Loss: 0.0221, Val Acc: 47.89%
Epoch [6/100], Train Loss: 0.0216, Train Acc: 49.83%, Val Loss: 0.0212, Val Acc: 50.46%
Epoch [7/100], Train Loss: 0.0205, Train Acc: 52.72%, Val Loss: 0.0196, Val Acc: 54.93%
Epoch [8/100], Train Loss: 0.0193, Train Acc: 55.59%, Val Loss: 0.0193, Val Acc: 56.28%
Epoch [9/100], Train Loss: 0.0183, Train Acc: 58.16%, Val Loss: 0.0180, Val Acc: 59.16%
Epoch [10/100], Train Loss: 0.0173, Train Acc: 60.64%, Val Loss: 0.0173, Val Acc: 60.54%
Epoch [11/100], Train Loss: 0.0164, Train Acc: 62.70%, Val Loss: 0.0165, Val Acc: 62.84%
Epoch [12/100], Train Loss: 0.